# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298606953699                   -0.85    4.1
  2   -8.300225008064       -2.79       -1.25    1.5
  3   -8.300446793564       -3.65       -1.87    3.4
  4   -8.300462026927       -4.82       -2.71    2.1
  5   -8.300464351817       -5.63       -3.20    3.6
  6   -8.300464493979       -6.85       -3.32    8.8
  7   -8.300464557469       -7.20       -3.45    1.0
  8   -8.300464600292       -7.37       -3.58    1.0
  9   -8.300464634053       -7.47       -3.80    1.0
 10   -8.300464640452       -8.19       -3.95    2.6
 11   -8.300464643516       -8.51       -4.31    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64206460917                   -0.70    5.1
  2   -16.64602126836       -2.40       -1.14    1.6
  3   -16.67923169229       -1.48       -1.86    4.5
  4   -16.67928264135       -4.29       -2.69    2.0
  5   -16.67928605209       -5.47       -3.02    6.4
  6   -16.67928620369       -6.82       -3.33    1.6
  7   -16.67928622020       -7.78       -3.76    1.1
  8   -16.67928622382       -8.44       -4.41    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31626675269                   -0.56    6.2
  2   -33.32365148917       -2.13       -1.00    1.2
  3   -33.33414381489       -1.98       -1.68    5.1
  4   -33.33481480673       -3.17       -2.39    2.0
  5   -33.33693437026       -2.67       -2.61    5.0
  6   -33.33693990432       -5.26       -2.85    6.1
  7   -33.33694383601       -5.41       -3.68    2.2
  8   -33.33694392161       -7.07       -3.94   12.0
  9   -33.33694392472       -8.51       -4.51    1.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298528235060                   -0.85    4.6
  2   -8.300260999562       -2.76       -1.58    1.6
  3   -8.300233738974   +   -4.56       -2.06    9.6
  4   -8.300366479727       -3.88       -2.25    4.5
  5   -8.300464324566       -4.01       -3.61    1.0
  6   -8.300464532666       -6.68       -3.80    5.6
  7   -8.300464639401       -6.97       -4.38    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32545239144                   -0.56    6.2
  2   -33.32793662034       -2.60       -1.28    1.0
  3   -24.96307284954   +    0.92       -0.61    7.2
  4   -33.21982507715        0.92       -1.51    9.0
  5   -33.32788511291       -0.97       -1.75    3.2
  6   -33.29444786380   +   -1.48       -1.62    3.5
  7   -33.05756306261   +   -0.63       -1.34    4.8
  8   -33.33095349303       -0.56       -2.10    4.5
  9   -33.33529935472       -2.36       -2.42    2.1
 10   -33.33618293669       -3.05       -2.36    2.4
 11   -33.33672140387       -3.27       -2.78    1.9
 12   -33.33688464985       -3.79       -3.07    3.2
 13   -33.33692787403       -4.36       -3.34    3.2
 14   -33.33694096068       -4.88       -3.67    2.8
 15   -33.33694214557       -5.93       -3.87    3.1
 16   -33.33694358877       -5.84       -4.18    1.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.